# **Modelo de Recomendación de Juegos (*item-item*)**

In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

Traemos los datasets necesarios:

In [3]:
df_genre = pd.read_parquet("./datasets_endpoints/games_genre_dummies.parquet")
df_genre

,app_name,release_date,price,item_id,developer,Accounting,Action,Adventure,Animation &amp; Modeling,Audio Production,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
0,Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,0,1,0,0,0,...,0,0,0,1,0,0,1,0,0,0
1,Ironbound,2018-01-04,free,643980,Secret Level SRL,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
2,Real Pool 3D - Poolians,2017-07-24,free,670290,Poolians.com,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
3,弹炸人2222,2017-12-07,0.99,767400,彼岸领域,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Battle Royale Trainer,2018-01-04,3.99,772540,Trickjump Games Ltd,0,1,1,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32127,Kebab it Up!,2018-01-04,1.99,745400,Bidoniera Games,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
32128,Colony On Mars,2018-01-04,1.99,773640,"Nikita ""Ghost_RUS""",0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
32129,LOGistICAL: South Africa,2018-01-04,4.99,733530,Sacada,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
32130,Russian Roads,2018-01-04,1.99,610660,Laush Dmitriy Sergeevich,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0


In [ ]:
#df_merged = df_games.merge(df_genres, on='game_id', how='left')

features = ['release_year'] + list(df_genre.columns[1:]) 

scaler = StandardScaler()
df_genre['release_year'] = scaler.fit_transform(df_genre[['release_year']])

df_final = df_genre[['game_id', 'app_name'] + features]

df_sampled = df_final.sample(n=2000, random_state=42)

save_path = '../data/processed/preprocessed_sample_for_recommendation.parquet'
df_sampled.to_parquet(save_path, compression='snappy')